# <u>Emotion detection from facial expressions
The aim of the problem is to find the emotion of a person by looking at the facial expression of that person. <br>The dataset used for this problem is from the Kaggle Challenge [Challenges in Representation Learning: Facial Expression Recognition Challenge](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data)

The task is to categorize each face based on the emotion shown in the facial expression in to one of seven categories:<br> 
- 0: Angry 
- 1:Disgust
- 2: Fear
- 3: Happy
- 4: Sad
- 5: Surprise
- 6: Neutral

In [1]:
# load the required modules
import pandas as pd
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import cv2

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras import callbacks
from keras.callbacks import EarlyStopping
from keras.layers import MaxPool2D
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

# <u>Data Preparation
The training set consists of 28,709 examples.
<br>The test set consists of 3,589 examples. 
<br>The cross validation set consists of another 3,589 examples.
There is a csv file containing the emotion values for all the data.

In [ ]:
# load the images
images_train = np.load('dataset/train_raw.npy')

: 

In [ ]:
# load the cross validation images
images_cv = np.load('dataset/cv_raw.npy')
# load the test images
images_test = np.load('dataset/test_raw.npy')

: 

In [ ]:
# load the corresponding emotion values
emotions = pd.read_csv('dataset/emotions.csv', header=None)

: 

In [ ]:
print(len(images_train))
print(len(images_cv))
print(len(images_test))
print(len(emotions))

: 

### Data distribution

In [ ]:
# data distribution
emotions.hist()

: 

In [ ]:
# distribution of pictures
emotions[0].value_counts()

: 

### Prepare output labels

In [ ]:
# prepare output labels for train, cv and test
y_train = emotions[:images_train.shape[0]].values
y_cv = emotions[images_train.shape[0]:images_train.shape[0] + images_cv.shape[0]].values
y_test = emotions[images_train.shape[0] + images_cv.shape[0]: ].values

: 

In [ ]:
print(y_train.shape)
print(y_cv.shape)
print(y_test.shape)

: 

In [ ]:
# create one hot encoding
y_train_ohe = to_categorical(y_train, num_classes=7)
y_cv_ohe = to_categorical(y_cv, num_classes=7)
y_test_ohe = to_categorical(y_test, num_classes=7)

: 

In [ ]:
print(y_train_ohe.shape)
print(y_cv_ohe.shape)
print(y_test_ohe.shape)

: 

### Sanity checking 

In [ ]:
# for displaying an image from dataset
def show(img):
    # resize the image
    show_image = img.reshape(48,48)
    # show the image
    plt.imshow(show_image, cmap='gray')

: 

In [ ]:
# verify the image with its label
index = 0
# training image
show(images_train[index])
print('Emotion:' + str(y_train[index]))

: 

In [ ]:
# cv image
show(images_cv[index])
print('Emotion:' + str(y_cv[index]))

: 

In [ ]:
# test image
show(images_test[index])
print('Emotion:' + str(y_test[index]))

: 

# <u>Preprocessing

In [ ]:
# reshape the images for keras model
images_train = np.expand_dims(images_train, 3)
images_cv = np.expand_dims(images_cv, 3)
images_test = np.expand_dims(images_test, 3)

: 

In [ ]:
print(images_train.shape)
print(images_cv.shape)
print(images_test.shape)

: 

# <u>Model Architecture

In [ ]:
from keras.utils import plot_model
import pydot
plot_model(model, to_file='drive/Colab Notebooks/model.png', show_shapes=True)

: 

In [ ]:
# save the model weights after each epoch if the validation loss decreases
checkpoint = ModelCheckpoint(filepath='models/weights_best.hdf5', verbose=1, save_best_only=True)

: 

In [ ]:
# load already saved model if needed
if os.path.exists('models/model.h5'):
    model = load_model('models/model.h5')
else:
    print('No model to load !')

: 

### Make the model

In [ ]:
model = Sequential()

# 1st stage
model.add(Conv2D(32, 3, input_shape=(48, 48, 1), padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

# 2nd stage
model.add(Conv2D(64, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.25))

# 3rd stage
model.add(Conv2D(128, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, 3, padding='same', 
                 activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.25))

# FC layers
model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(7))
model.add(Activation('softmax'))

: 

In [ ]:
model.summary()

: 

In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

: 

### NOTE: The model was trained for 50 epochs but was done not in one go, so in the fit function there is only 5 written as it was the last time the model was run

In [ ]:
# start training the model
hist = model.fit(images_train, y_train_ohe, batch_size=128, epochs=5, verbose=1,
          validation_data=(images_cv, y_cv_ohe), shuffle=True, callbacks=[checkpoint])

# save the current model and weights
model.save_weights('models/weights.h5')
model.save('models/model.h5')

: 

## <u>Model Evaluation
The model achieves 63% accuracy on the unseen test set.

In [ ]:
# training loss and accuracy
train_acc = hist.history['acc']
val_acc = hist.history['val_acc']
print('Training Accuracy: ', train_acc[-1])
print('Validation Accuracy: ', val_acc[-1])

# test loss and accuracy
score, acc = model.evaluate(images_test, y_test_ohe,
                            batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

: 

### Plot the training loss and accuracy graph

In [ ]:
plt.figure(figsize=(16,5))
# training loss graph
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(hist.history['loss'], color='b', label='Training Loss')
plt.plot(hist.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

# training accuracy graph
plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(hist.history['acc'], color='b', label='Training Accuracy')
plt.plot(hist.history['val_acc'], color='r', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()



: 

## Let us do prediction on some faces

In [ ]:
# read image 
images = []
for filename in os.listdir('test_images\\'):
    path = os.path.join('test_images\\', filename)
    images.append(cv2.imread(path, -1))    

: 

In [ ]:
# do prediction for the images
predictions = []
for img in images:
    # change to greyscale
    curr_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    curr_img = cv2.resize(curr_img, (48,48))
    curr_img = np.reshape(curr_img, (1, 48, 48, 1))
    predictions.append(np.argmax(model.predict(curr_img)))

: 

In [ ]:
# list of given emotions
EMOTIONS = ['Angry', 'Disgusted', 'Fearful',
            'Happy', 'Sad', 'Surprised', 'Neutral']

: 

In [ ]:
# bgr to rgb
for i in range(5):
    images[i] = cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB)

: 

In [ ]:
plt.imshow(images[0])
print('Predicted Emotion: ' + str(EMOTIONS[predictions[0]]))

: 

In [ ]:
plt.imshow(images[1])
print('Predicted Emotion: ' + str(EMOTIONS[predictions[1]]))

: 

In [ ]:
plt.imshow(images[2])
print('Predicted Emotion: ' + str(EMOTIONS[predictions[2]]))

: 

In [ ]:
plt.imshow(images[3])
print('Predicted Emotion: ' + str(EMOTIONS[predictions[3]]))

: 

In [ ]:
plt.imshow(images[4])
print('Predicted Emotion: ' + str(EMOTIONS[predictions[4]]))

: 

: 